In [1]:
year=int(input())
year1=str(year-1911)
year2=str(year)
month=str(input())
day=str(input())
d1=year-1911-1
import requests
from io import StringIO
import pandas as pd
import numpy as np
link = 'http://www.tpex.org.tw/web/stock/aftertrading/daily_close_quotes/stk_quote_download.php?l=zh-tw&d=' + year1 + '/' + month + '/' + day + '&s=0,asc,0'
r = requests.get(link)
r.ok
lines = r.text.replace('\r', '').split('\n')
df1 = pd.read_csv(StringIO("\n".join(lines[3:])), header=None)
df1.columns = list(map(lambda l: l.replace(' ',''), lines[2].split(',')))
df1=df1[['代號','名稱','收盤','成交股數']]
r = requests.post('https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + year2 + month + day + '&type=ALL')
df2 = pd.read_csv(StringIO(r.text.replace("=", "")), 
            header=["證券代號" in l for l in r.text.split("\n")].index(True)-1, encoding='gbk')
df2=df2[['證券代號','證券名稱','收盤價','成交股數']]
df2=df2.rename(columns={'證券代號':'代號'})
df2=df2.rename(columns={'證券名稱':'名稱'})
df2=df2.rename(columns={'收盤價':'收盤'})
收盤=pd.concat([df1,df2], ignore_index=True)
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
def 財務比率(year,dd,aa):
    if dd==1:
        url = 'https://mops.twse.com.tw/mops/web/t51sb02'
        if aa==1:
            form_data = {
                    'encodeURIComponent':1,
                    'run':'Y',
                    'step':1,
                    'TYPEK':'sii', #otc
                    'year': year,
                    'firstin':1,
                    'off':1,
                    'ifrs':'Y',
                }

        if aa==2:
            form_data = {
                    'encodeURIComponent':1,
                    'run':'Y',
                    'step':1,
                    'TYPEK':'otc', #otc
                    'year': year,
                    'firstin':1,
                    'off':1,
                    'ifrs':'Y',
                }


        data = get_data_from_mops_twse(url, form_data)
        df = pd.DataFrame(np.array(data),
                          columns=['Number', 
                                   'Name', 
                                   'DebtAssetRatio', 
                                   'LongTermFundsRealEstateRatio', 
                                   'CurrentRatio', 
                                   'QuickCurrentRatio', 
                                   'InterestProtectionMutiple',
                                   'AccountsReceivableTurnover',
                                   'AverageDaysOfCashCollection',
                                   'InventoryTurnoverRate',
                                   'AverageSalesDays',
                                   'AverageSalesRealEstateTurover',
                                   'TotalAssetsWeekTurnover',
                                   'ROA',
                                   'ROE',
                                   'NetProfitBeforeTaxAccountForPaid-inCapitalTheRatio',
                                   'NetProfitRatio',
                                   'EPS',
                                   'CashFlowRatio',
                                   'CashFlowAllowableRatio',
                                   'CashReinvestmentCapitalRatio']
                          )
    if dd==2:
        url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb06'
        if aa == 1:
            form_data = {
                    'encodeURIComponent': 1,
                    'step':1,
                    'firstin':1,
                    'off':1,
                    'isQuery':'Y',
                    'TYPEK':'sii',
                    'year':year, 
                    'season':'04', 
                }
        if aa == 2:
            form_data = {
                    'encodeURIComponent': 1,
                    'step':1,
                    'firstin':1,
                    'off':1,
                    'isQuery':'Y',
                    'TYPEK':'otc',
                    'year':year, 
                    'season':'04', 
                }

        data = get_data_from_mops_twse(url, form_data)
        df = pd.DataFrame(np.array(data),
                          columns=['Number', 
                               'Name', 
                               'OperatingIncome', 
                               'GrossMargin', 
                               'OperatingProfitRatio', 
                               'NetProfitRateBeforeTax', 
                               'NetProfitAfterTax',
                               ]
                          )
    
    return df

def get_data_from_mops_twse(url, form_data):
    response = requests.post(url,form_data)
    response.encoding = 'utf8'
    text = response.text
    soup = BeautifulSoup(text, 'html.parser')    
    tables = soup.find_all("table", {"class" : "hasBorder"})
    data = []
    for tb in tables:
        for row in tb.find_all("tr"):
            #print(row.text)
            tempdata = []
            for col in row.find_all('td'):
                col.attrs = {}
                #print(col.text.strip().replace('\u3000', ''))
                tempdata.append(col.text.strip().replace('\u3000', ''))
            if len(tempdata) <= 1:
                pass 
            else:
                data.append(tempdata)
    
    return data
import time
營益市={}
營益櫃={}
財務市={}
財務櫃={}
if int(month)<=3:
    a=[d1-1,d1-2,d1-3,d1-4,d1-5,d1-6]
else:
    a=[d1,d1-1,d1-2,d1-3,d1-4,d1-5]
for i in a:
    營益市[i]=財務比率(i,2,1)
    營益櫃[i]=財務比率(i,2,2)
    財務市[i]=財務比率(i,1,1)
    財務櫃[i]=財務比率(i,1,2)
    time.sleep(5)
營益={}
財務={}
for i in a:
    營益[i]=pd.concat([營益市[i],營益櫃[i]], ignore_index=True)
    財務[i]=pd.concat([財務市[i],財務櫃[i]], ignore_index=True)
for i in a:
    營益[i].columns=營益[i].columns+str(i)
    財務[i].columns=財務[i].columns+str(i)
    營益[i]=營益[i].rename(columns={'Number'+str(i):'代號'}).rename(columns={'Name'+str(i):'名稱'})
    財務[i]=財務[i].rename(columns={'Number'+str(i):'代號'}).rename(columns={'Name'+str(i):'名稱'})
for i in a:
    營益[i]=營益[i].drop_duplicates('代號')
    財務[i]=財務[i].drop_duplicates('代號')
    營益[i]=營益[i][(營益[i]['代號'] != '公司代號')]
from functools import reduce
if int(month)<=3:
    listdf1 = [營益[d1-1],營益[d1-2],營益[d1-3],營益[d1-4],營益[d1-5],營益[d1-6],財務[d1-1],財務[d1-2],財務[d1-3],財務[d1-4],財務[d1-5],財務[d1-6]]
else:
    listdf1 = [營益[d1],營益[d1-1],營益[d1-2],營益[d1-3],營益[d1-4],營益[d1-5],財務[d1],財務[d1-1],財務[d1-2],財務[d1-3],財務[d1-4],財務[d1-5]]
財務比率 = reduce(lambda  left,right: pd.merge(left,right,on=['代號', '名稱'], how='inner'), listdf1)

def monthly_report(year, month, aa):
    if aa == 1:
        url = 'https://mops.twse.com.tw/nas/t21/sii/t21sc03_'+str(year)+'_'+str(month)+'_0.html'
    if aa == 2:
        url = 'https://mops.twse.com.tw/nas/t21/otc/t21sc03_'+str(year)+'_'+str(month)+'_0.html'
    if aa == 3:
        url = 'https://mops.twse.com.tw/nas/t21/sii/t21sc03_'+str(year)+'_'+str(month)+'_1.html'
    if aa == 4:
        url = 'https://mops.twse.com.tw/nas/t21/otc/t21sc03_'+str(year)+'_'+str(month)+'_1.html'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    r = requests.get(url, headers=headers)
    r.encoding = 'big5'

    dfs = pd.read_html(StringIO(r.text), encoding='big-5')
    group = pd.concat(df for df in dfs if df.shape[1] == 2).columns.to_list()
    group = [s for s in group if '產業別：' in s]
    group = [w.replace('產業別：', '') for w in group]
    stock = list([df for df in dfs if df.shape[1] <= 11 and df.shape[1] > 5])
    listdf=[]
    for i in range(len(group)):
        stock[i]['產業']=group[i]
        listdf.append(stock[i])
    df=pd.concat(listdf)
    

    #df = pd.concat([df for df in dfs if df.shape[1] <= 11 and df.shape[1] > 5])
    
    if 'levels' in dir(df.columns):
        df.columns = df.columns.get_level_values(1)
    else:
        df = df[list(range(0,10))]
        column_index = df.index[(df[0] == '公司代號')][0]
        df.columns = df.iloc[column_index]
    
    df['當月營收'] = pd.to_numeric(df['當月營收'], 'coerce')
    df = df[~df['當月營收'].isnull()]
    df = df[df['公司代號'] != '合計']
    df = df.rename(columns={'':'產業別'})
    return df

import time
if int(month)>2:
    if int(day)<10:
        月營收市=monthly_report(d1+1,int(month)-2,1)
        月營收櫃=monthly_report(d1+1,int(month)-2,2)
        月營收外市=monthly_report(d1+1,int(month)-2,3)
        月營收外櫃=monthly_report(d1+1,int(month)-2,4)
    if int(day)>=10:
        月營收市=monthly_report(d1+1,int(month)-1,1)
        月營收櫃=monthly_report(d1+1,int(month)-1,2)
        月營收外市=monthly_report(d1+1,int(month)-1,3)
        月營收外櫃=monthly_report(d1+1,int(month)-1,4)
if int(month)==1:
    if int(day)<10:
        月營收市=monthly_report(d1,11,1)
        月營收櫃=monthly_report(d1,11,2)
        月營收外市=monthly_report(d1,11,3)
        月營收外櫃=monthly_report(d1,11,4)
    if int(day)>=10:
        月營收市=monthly_report(d1,12,1)
        月營收櫃=monthly_report(d1,12,2)
        月營收外市=monthly_report(d1,12,3)
        月營收外櫃=monthly_report(d1,12,4)
if int(month)==2:
    if int(day)<10:
        月營收市=monthly_report(d1,12,1)
        月營收櫃=monthly_report(d1,12,2)
        月營收外市=monthly_report(d1,12,3)
        月營收外櫃=monthly_report(d1,12,4)
    if int(day)>=10:
        月營收市=monthly_report(d1+1,1,1)
        月營收櫃=monthly_report(d1+1,1,2)
        月營收外市=monthly_report(d1+1,1,3)
        月營收外櫃=monthly_report(d1+1,1,4)
月財報=pd.concat([月營收市,月營收櫃,月營收外市,月營收外櫃], ignore_index=True)
月財報=月財報.rename(columns={'公司代號':'代號'}).rename(columns={'公司名稱':'名稱'})
from functools import reduce
listdf1 = [月財報,財務比率,收盤]
報表 = reduce(lambda  left,right: pd.merge(left,right,on='代號', how='inner'), listdf1)
if int(month)<=3:
    list1=['代號','名稱','收盤','前期比較增減(%)','成交股數','產業別']
else:
    list1=['代號','名稱','收盤','前期比較增減(%)','成交股數','產業別']
for i in a:
    list1.append('ROE'+str(i))
    list1.append('EPS'+str(i))
    list1.append('OperatingProfitRatio'+str(i))
    list1.append('OperatingIncome'+str(i))
報表=報表[list1]
for i in a:
    if int(month)<=3:
        報表=報表.rename(columns={'EPS'+str(i):'EPS'+str(i-d1)})
        報表=報表.rename(columns={'ROE'+str(i):'ROE'+str(i-d1)})
        報表=報表.rename(columns={'OperatingProfitRatio'+str(i):'營業利益率'+str(i-d1)})
        報表=報表.rename(columns={'OperatingIncome'+str(i):'營業收入'+str(i-d1)})
    else:
        報表=報表.rename(columns={'EPS'+str(i):'EPS'+str(i-d1-1)})
        報表=報表.rename(columns={'ROE'+str(i):'ROE'+str(i-d1-1)})
        報表=報表.rename(columns={'OperatingProfitRatio'+str(i):'營業利益率'+str(i-d1-1)})
        報表=報表.rename(columns={'OperatingIncome'+str(i):'營業收入'+str(i-d1-1)})
報表=報表.dropna(axis=0)
i = 報表[報表.values == 'NA'].index
j = 報表[報表.values == '--'].index
k = 報表[報表.values == ' ---'].index
yy = 報表[報表.values == '---'].index 
bb = 報表[報表.values == '----'].index
kk = 報表[報表.values == ''].index
c=i.append(j).append(k).append(yy).append(bb).append(kk)
c=c.drop_duplicates(keep='last')
報表=報表.drop(c)
for col in ['EPS-1', 'EPS-2', 'EPS-3','EPS-4','EPS-5','ROE-1','ROE-2', 'ROE-3','ROE-4','ROE-5','收盤','營業利益率-1','營業利益率-2','營業利益率-3','營業利益率-4','營業利益率-5','營業利益率-6','營業收入-1','營業收入-2','營業收入-3','營業收入-4','營業收入-5','營業收入-6','前期比較增減(%)','成交股數']:
    報表[col]=報表[col].astype(str).str.replace(',', '').astype(float)
報表['平均EPS']=(報表['EPS-1']+報表['EPS-2']+報表['EPS-3']+報表['EPS-4']+報表['EPS-5'])/5
報表['本益比']=報表['收盤']/報表['平均EPS']
for s in range(1,7):
    報表['營業利益-'+str(s)]=(報表['營業利益率-'+str(s)]*報表['營業收入-'+str(s)])/100
#報表=報表[(報表['營業利益-1']>報表['營業利益-2'])&(報表['營業利益-2']>報表['營業利益-3'])&(報表['營業利益-3']>報表['營業利益-4'])&(報表['營業利益-4']>報表['營業利益-5'])]
for col in ['營業利益-1','營業利益-2','營業利益-3','營業利益-4','營業利益-5','營業利益-6']:
    報表=報表[報表[col]>0]
報表['營業利益-growth-rate1']=(報表['營業利益-1']/報表['營業利益-2'])-1
報表['營業利益-growth-rate2']=(報表['營業利益-2']/報表['營業利益-3'])-1
報表['營業利益-growth-rate3']=(報表['營業利益-3']/報表['營業利益-4'])-1
報表['營業利益-growth-rate4']=(報表['營業利益-4']/報表['營業利益-5'])-1
報表['營業利益-growth-rate5']=(報表['營業利益-5']/報表['營業利益-6'])-1
import math
報表['平均年成長率']=報表[['營業利益-growth-rate1','營業利益-growth-rate2','營業利益-growth-rate3','營業利益-growth-rate4','營業利益-growth-rate5']].median(axis=1)
報表['本益成長比']=報表['本益比']/(報表['平均年成長率']*100)
#for col in ['ROE-1','ROE-2','ROE-3','ROE-4','ROE-5']:
#    報表=報表[(報表[col]>8)]
#報表=報表[(報表['前期比較增減(%)']>0)]
#報表=報表[(報表['營業利益率-1']>10)]
報表=報表[(報表['成交股數']>1000000)]
報表=報表[(報表['本益比']>0)&(報表['平均年成長率']>0)]
#報表=報表[報表['產業別']=='食品工業']
#a=pd.read_csv("C:/Users/admun/Desktop/活頁簿1.csv",engine='python')
#a=a.rename(columns={'id':'代號'})
#報表['代號']=報表['代號'].astype(float)
#報表=pd.merge(a,報表,on='代號', how='inner')



def 資產(year,d,aa):            
    if d == 4:
        url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb04'
    elif d == 5:
        url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb05'
    elif d == 20:
        url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb20'
    else:
        print('type does not match')
    if aa == 1:
        payload={
             'encodeURIComponent': 1,
             'step':1,
             'firstin':1,
             'off':1,
             'isQuery':'Y',
             'TYPEK':'sii',
             'year':year, 
             'season':'04', 
             }
    if aa == 2:
        payload={
             'encodeURIComponent': 1,
             'step':1,
             'firstin':1,
             'off':1,
             'isQuery':'Y',
             'TYPEK':'otc',
             'year':year, 
             'season':'04', 
             }
    headers = {'Host': 'mops.twse.com.tw','User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36'}
    list_req = requests.post(url=url, data=payload ,headers=headers)
    soup = BeautifulSoup(list_req.content, "html.parser")
    all_soup = soup.findAll('tr')
    header_soup = soup.findAll('tr',class_="tblHead")
    row_soup = soup.findAll('tr', class_=['even','odd'])
    i=0
    r=0
    a=[]
    b=[]
    for h in all_soup:
        r=r+1
        if h in header_soup:
            i=i+1
            a.append(r)
            b.append(i)
    a.append(100000)
    asset_data={}
    for s in range(len(b)):
        headers = []
        y=b[s]-1
        for h in header_soup[y]:
            headers.append(h.text)
        rows=[]
        x=a[s]
        z=a[s+1]
        for row in set(all_soup[x:z]).intersection(set(row_soup)):
            row_data = []
      
            for cell in row.findAll('td'):
                row_data.append(cell.text)
            rows.append(row_data)
        
        
    
        asset_data[s]= pd.DataFrame(rows, columns=headers)  
    listdf=[]
    for j in range(s+1):
        listdf.append(asset_data[j])
    asset市=pd.concat(listdf)
    return asset市

if int(month)<=3:
    d2=d1-1
else:
    d2=d1
資產市=資產(d2,5,1)
資產櫃=資產(d2,5,2)
資產=pd.concat([資產市,資產櫃], ignore_index=True)
col1=['公司代號','公司名稱','每股參考淨值']
資產=資產[col1]
資產=資產.rename(columns={'公司代號':'代號'})
報表=pd.merge(資產,報表,on='代號', how='inner')
報表['每股參考淨值']=報表['每股參考淨值'].astype(str).str.replace(',', '').astype(float)
報表['股淨比']=報表['收盤']/報表['每股參考淨值']
col2=['代號','公司名稱','收盤','產業別','本益成長比','股淨比']
報表=報表[col2]
報表=報表[報表['股淨比']<1]
報表=報表[報表['本益成長比']<0.7]
報表=報表.sort_values(['股淨比'], ascending=True)
報表


2023
02
22


,代號,公司名稱,收盤,產業別,本益成長比,股淨比
49,1312,國喬,19.85,塑膠工業,0.336159,0.517601
164,5512,力麒,10.35,建材營造,0.318921,0.565574
32,2104,國際中橡,20.85,橡膠工業,0.325392,0.604348
143,2851,中再保,18.45,金融保險業（其中金控公司係控股公司，其申報之「營業收入」係認列所有子公司損益之合計數）,0.409048,0.657989
7,2601,益航,8.60,貿易百貨,0.424829,0.722689
87,6116,彩晶,13.10,光電業,0.288110,0.758102
55,9945,潤泰新,44.20,其他,0.605114,0.794107
60,2534,宏盛,23.20,建材營造,0.461258,0.815179
31,1907,永豐餘,26.10,造紙工業,0.320055,0.846304
46,1909,榮成,18.65,造紙工業,0.437017,0.873127
